In [1]:
import random
import torch
import numpy as np
import pandas as pd

In [2]:
from models.neural_networks.utils import prepare_data_for_predict
from features.build_features import calculate_expenses
from data.utils import load_default_data
from features.build_features import aggregate_users_data
from models.neural_networks.NeuralNetworkRegressor import NeuralNetworkRegressor
from models.neural_networks.SimpleRegressor import SimpleRegressor
from models.neural_networks.utils import train

In [3]:
seed = 213769420
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [4]:
train_sessions, test, products, users, deliveries = load_default_data(prefix="../../data/")
targets = calculate_expenses(test, products, users)
targets

,user_id,expenses
0,102,0.00
1,103,554.77
2,104,2332.01
3,105,0.00
4,106,0.00
...,...,...
195,297,109.00
196,298,2399.00
197,299,0.00
198,300,0.00


In [5]:
targets['expenses'].mean()

656.25255

In [6]:
# targets['expenses'] = targets['expenses'].apply(lambda b: 1 if b > 5000 else 0) binarization
targets['expenses'] = targets['expenses'].apply(lambda x: x/100)

In [7]:
some_net = SimpleRegressor()
users_data = aggregate_users_data(train_sessions, users, products)

In [8]:
train(some_net, users_data, targets)

Epoch 0 loss nan
Epoch 1 loss nan
Epoch 2 loss nan
Epoch 3 loss nan
Epoch 4 loss nan
Epoch 5 loss nan
Epoch 6 loss nan
Epoch 7 loss nan
Epoch 8 loss nan
Epoch 9 loss nan
Epoch 10 loss nan
Epoch 11 loss nan
Epoch 12 loss nan
Epoch 13 loss nan
Epoch 14 loss nan
Epoch 15 loss nan
Epoch 16 loss nan
Epoch 17 loss nan
Epoch 18 loss nan
Epoch 19 loss nan
Epoch 20 loss nan
Epoch 21 loss nan
Epoch 22 loss nan
Epoch 23 loss nan
Epoch 24 loss nan
Epoch 25 loss nan
Epoch 26 loss nan
Epoch 27 loss nan
Epoch 28 loss nan
Epoch 29 loss nan
Epoch 30 loss nan
Epoch 31 loss nan
Epoch 32 loss nan
Epoch 33 loss nan
Epoch 34 loss nan
Epoch 35 loss nan
Epoch 36 loss nan
Epoch 37 loss nan
Epoch 38 loss nan
Epoch 39 loss nan
Epoch 40 loss nan
Epoch 41 loss nan
Epoch 42 loss nan
Epoch 43 loss nan
Epoch 44 loss nan
Epoch 45 loss nan
Epoch 46 loss nan
Epoch 47 loss nan
Epoch 48 loss nan
Epoch 49 loss nan
Epoch 50 loss nan
Epoch 51 loss nan
Epoch 52 loss nan
Epoch 53 loss nan
Epoch 54 loss nan
Epoch 55 loss nan
Ep

In [9]:
extracted_users_data = aggregate_users_data(train_sessions, users, products)
x, cat_x = prepare_data_for_predict(extracted_users_data)
x = torch.from_numpy(x.values).float()
cat_x = torch.from_numpy(cat_x.values).float()
some_net.eval()
out = some_net(x, cat_x).squeeze()
out = out.detach().numpy()

In [10]:
# out_array = {int(user_id): out[i] > 0.3 for i, user_id in enumerate(extracted_users_data["user_id"].to_list())} binarization
out_array = {int(user_id): out[i] for i, user_id in enumerate(extracted_users_data["user_id"].to_list())}

In [11]:
evaluation_array = {int(user_id): targets['expenses'].mean() for i, user_id in enumerate(extracted_users_data["user_id"].to_list())}

In [12]:
def loss(targets, out_array):
    sum = 0
    for row in targets.values[:]:
        sum+= (row[1] - out_array[row[0]])**2

    return sum

In [13]:
loss(targets, out_array)

nan

In [14]:
loss(targets, evaluation_array)

35484.379756659895

In [15]:
torch.save(some_net.state_dict(), "../parameters/simple_v1")